In [1]:
from PyAFAR_GUI.infant_afar import infant_afar

2025-02-11 14:04:48.304202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1739279110.558607   26797 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) Iris(TM) Plus Graphics 655
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1739279110.565465   27911 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739279110.573174   27911 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


~ 3min

In [2]:
import pandas as pd 
import os

In [3]:
folder_path = "../data/CityInfant/BlackWhite"

# List of files to remove
to_remove = ['Baby face 1.bmp', 'baby face 2.bmp', 'baby face 3.bmp', 'G33NegBW(1).bmp']

# Get all files in the folder
files = os.listdir(folder_path)

# Loop through files
for file in files:
    file_path = os.path.join(folder_path, file)

    if file in to_remove:
        os.remove(file_path)  # Remove unwanted files
        print(f"Deleted: {file}")

    elif file.startswith("Copy of "):  # Check if it's a copy
        original_file = file.replace("Copy of ", "", 1)  # Get the original filename
        original_path = os.path.join(folder_path, original_file)

        if original_file in files:
            os.remove(file_path)  # Delete the copy
            print(f"Deleted duplicate: {file}")
        else:
            new_path = os.path.join(folder_path, original_file)
            os.rename(file_path, new_path)  # Rename the copy
            print(f"Renamed {file} to {original_file}")


In [14]:
# Full AU
# AUs=["au_1", "au_2", "au_3","au_4", "au_6", "au_7","au_9", "au_10", "au_12","au_14", "au_15", "au_17","au_20","au_23", "au_24", "au_28"]

In [9]:
# Detect AU

def process_images_to_dataframe(folder_path):
    keys_of_interest = ["Pitch", "Yaw", "Roll", "Eye Aspect Ratio", "Mouth Aspect Ratio",
                        "Occ_au_1", "Occ_au_2", "Occ_au_3", "Occ_au_4",
                        "Occ_au_6", "Occ_au_9", "Occ_au_12", "Occ_au_20", "Occ_au_28"]

    data_list = []

    # Loop through each image in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".bmp"):
            image_path = os.path.join(folder_path, file)

            results = infant_afar(image_path, AUs=["au_1", "au_2", "au_3", "au_4", "au_6",
                                                   "au_9", "au_12", "au_20", "au_28"],
                                  GPU=True, max_frames=1000)
            
            # Create a dictionary with only the keys of interest
            filtered_results = {key: results[key][0] for key in keys_of_interest if key in results}
            filtered_results['Image'] = file
            data_list.append(filtered_results)

    # Convert list of dictionaries to a DataFrame
    df = pd.DataFrame(data_list)
    cols = ['Image'] + [col for col in df.columns if col != 'Image']
    df = df[cols]    
    return df

df_results = process_images_to_dataframe(folder_path)
#df_results

../data/CityInfant/BlackWhite/G1PosBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G35NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G34NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/B28NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/B29NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G21PosBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G20PosBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/B17NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G15PosBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/G14PosBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/B6NegBW.bmp
True
Length of video
-9223372036854775808
../data/CityInfant/BlackWhite/B7NegBW.bmp
True
Length of video
-922

In [45]:
# Match with imageID

metadata = pd.read_csv('../data/CityInfant/Validationinfo/FinalImages.csv')

metadata['imageCat'] = metadata['imageCat'].astype(int)

# Create lookup_key in metadata
metadata['lookup_key'] = metadata['imageDBName'] + metadata['imageCat'].map({1: 'Neg', 2: 'Neu', 3: 'Pos'})
metadata['lookup_key'] = metadata['lookup_key'].str.strip()

# Extract 'imageDBName' from the filename, ignoring case and handling optional numbers before BW
df_results['imageDBName'] = df_results['Image'].str.extract(r'([A-Za-z0-9]+)(?:\d*)?(Pos|Neg|Neu)[Bb][Ww]\.bmp')[0]

# Extract 'Pos', 'Neg', or 'Neu' from the filenames and map them to corresponding imageCat
df_results['imageCat'] = df_results['Image'].str.extract(r'(Pos|Neg|Neu)')[0].map({'Pos': 3, 'Neg': 1, 'Neu': 2})

# Create 'lookup_key' in df_results
df_results['lookup_key'] = df_results['imageDBName'] + df_results['Image'].str.extract(r'(Pos|Neg|Neu)')[0]

# Strip any extra whitespace
df_results['lookup_key'] = df_results['lookup_key'].str.strip()
metadata['lookup_key'] = metadata['lookup_key'].str.strip()

# Ensure both 'lookup_key' columns are strings
df_results['lookup_key'] = df_results['lookup_key'].astype(str)
metadata['lookup_key'] = metadata['lookup_key'].astype(str)

# Merge df_results with metadata to get 'imageID'
df = df_results.merge(metadata[['lookup_key', 'imageID']], on='lookup_key', how='left')

In [ ]:
print(df['imageID'].isna().sum())
nan_ids = df[df['imageID'].isna()]
#print(nan_ids)


In [62]:
# Assign manually imageID to remaining images

manual_ids = {
    'G27NewBW.bmp': 231 ,
    'G1Neu2BW.bmp': 145,
    'B23Neu1BW.bmp': 77,
    #3NeuBW.bmp': YOUR_ID_4, # Not in validation set
    'G1Neu1BW.bmp': 143,
    #'G32NeuBW.bmp': YOUR_ID_6, # Not in valdiation set 
    'B23Neu2BW.bmp': 78
}

# Assign the IDs manually
df.loc[df['Image'].isin(manual_ids.keys()), 'imageID'] = df['Image'].map(manual_ids)


In [63]:
#df[df['Image'].str.startswith('G32')]

In [64]:
df['imageCat'] = df['imageCat'].astype('Int64')
df['imageID'] = df['imageID'].astype('Int64')
df


,Image,Pitch,Yaw,Roll,Eye Aspect Ratio,Mouth Aspect Ratio,Occ_au_1,Occ_au_2,Occ_au_3,Occ_au_4,Occ_au_6,Occ_au_9,Occ_au_12,Occ_au_20,Occ_au_28,imageDBName,imageCat,lookup_key,imageID
0,G1PosBW.bmp,2.271588,1.503724,-0.020476,0.844543,0.564269,0.604962,0.156330,0.100698,0.751398,0.960786,0.053498,0.827365,0.914809,0.011352,G1,3,G1Pos,146
1,G35NegBW.bmp,9.595572,-1.007420,-0.075786,0.738996,0.566980,0.554940,0.247877,0.963012,0.892812,0.988188,0.622714,0.599941,0.992239,0.011972,G35,1,G35Neg,258
2,G34NegBW.bmp,11.969004,-10.082613,0.067473,0.888249,0.497927,0.425449,0.031160,0.643459,0.670010,0.244077,0.003154,0.004786,0.854814,0.111297,G34,1,G34Neg,255
3,B28NegBW.bmp,-6.011296,-18.061570,0.362115,0.742355,0.558649,0.276088,0.128461,0.990733,0.991031,0.953296,0.977597,0.051968,0.921999,0.002898,B28,1,B28Neg,95
4,B29NegBW.bmp,-11.146152,10.917997,0.343841,0.785101,0.515582,0.409443,0.127541,0.769169,0.861662,0.459491,0.002681,0.114230,0.542987,0.008241,B29,1,B29Neg,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,G33PosBW.bmp,11.690165,-1.346159,-0.057319,0.901336,0.514254,0.690283,0.461885,0.067139,0.023338,0.687406,0.000076,0.912756,0.462816,0.174139,G33,3,G33Pos,247
149,G18NegBW.bmp,1.493996,-1.002233,-0.017406,0.799191,0.518972,0.718797,0.861542,0.867889,0.981839,0.982293,0.987907,0.867165,0.947914,0.060732,G18,1,G18Neg,198
150,G19NegBW.bmp,-4.855926,7.234188,0.023119,0.758119,0.597042,0.801437,0.553560,0.976905,0.957925,0.949935,0.971254,0.303627,0.793386,0.000971,G19,1,G19Neg,202
151,G6NegBW.bmp,14.895807,3.109167,0.031230,0.800365,0.570024,0.925507,0.587296,0.825456,0.027568,0.039426,0.001254,0.002879,0.080728,0.476343,G6,1,G6Neg,160


In [65]:
df[df['imageID'].isna()]

,Image,Pitch,Yaw,Roll,Eye Aspect Ratio,Mouth Aspect Ratio,Occ_au_1,Occ_au_2,Occ_au_3,Occ_au_4,Occ_au_6,Occ_au_9,Occ_au_12,Occ_au_20,Occ_au_28,imageDBName,imageCat,lookup_key,imageID
106,G3NeuBW.bmp,1.049271,3.067426,0.023518,0.876726,0.513867,0.468831,0.452560,0.053664,0.07211,0.003907,0.000032,0.003576,0.058262,0.002414,G3,2,G3Neu,<NA>
142,G32NeuBW.bmp,5.702969,-0.456867,-0.033979,0.844677,0.515330,0.828512,0.695525,0.476174,0.27148,0.845113,0.002015,0.160547,0.605559,0.066148,G32,2,G32Neu,<NA>


In [70]:
# Save clean csv 

df_cleaned = df.dropna(subset=['imageID'])

df_cleaned = df_cleaned.sort_values(by='imageID')
df_cleaned = df_cleaned[['imageID'] + [col for col in df_cleaned.columns if col != 'imageID']]
print(df_cleaned)

df_cleaned.to_csv('../data/AU/pyafar_infants.csv', index=False)


     imageID         Image      Pitch        Yaw      Roll  Eye Aspect Ratio  \
69         1   B1NegBW.bmp  -1.022183   6.189266  0.142265          0.868574   
136        2   B1PosBw.bmp  -0.289924   0.289790 -0.012340          0.862333   
117        3   B2NegBw.bmp -13.230544   2.844317 -0.038627          0.746718   
46         4   B2NeuBW.bmp   0.777842   3.026587 -0.019514          0.873425   
41         6   B2PosBW.bmp  -3.847174  -0.709194 -0.027637          0.821899   
..       ...           ...        ...        ...       ...               ...   
2        255  G34NegBW.bmp  11.969004 -10.082613  0.067473          0.888249   
80       256  G34NeuBW.bmp   9.400520  -4.558382  0.034506          0.911669   
87       257  G34PosBW.bmp  -2.238079  -0.327019 -0.039003          0.833275   
1        258  G35NegBW.bmp   9.595572  -1.007420 -0.075786          0.738996   
86       259  G35PosBW.bmp -10.930387   7.578057  0.073111          0.831003   

     Mouth Aspect Ratio  Occ_au_1  Occ_